## Import of libraries

In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

## Import of CSV data

In [9]:
hashed_df = pd.read_csv('hashed_feature.csv')
tabular_df = pd.read_csv('tabular_data.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
tabular_df.head(3)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,1,110.0,55.0,0.432017,0.0,176.78,0.0,0.323712,NaN,...,0.00,0,0.0,0.0,55.0,2.0,0.526552,145.0,133.28,0.0
1,0,2,110.0,110.0,0.397517,0.0,315.42,0.0,0.316798,NaN,...,0.00,0,0.0,0.0,110.0,1.0,0.481063,130.0,229.97,0.0
2,0,3,110.0,55.0,0.359440,0.0,354.55,0.0,0.339188,NaN,...,0.07,0,0.0,0.0,55.0,1.0,0.509598,180.0,231.78,0.0


## Simplest solution to deal with NaN 

In [10]:
tabular_df = tabular_df.fillna(axis='columns', method='bfill')
tabular_df.head(3)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,1,110,55,0.432017,0,176.78,0,0.323712,147,...,0,0,0,0,55,2,0.526552,145,133.28,0
1,0,2,110,110,0.397517,0,315.42,0,0.316798,135,...,0,0,0,0,110,1,0.481063,130,229.97,0
2,0,3,110,55,0.35944,0,354.55,0,0.339188,181,...,0.07,0,0,0,55,1,0.509598,180,231.78,0


## Create smaller DF based on 4 columns to test different models and check if its even possible to apply them for this task

In [11]:
simp_tab_df = tabular_df[['id', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6']].copy()
simp_tab_df = simp_tab_df.convert_dtypes()

In [12]:
simp_tab_df.head(14)

,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,0,55.00,0.432017,0.000000,176.78,0.0,0.323712
1,0,110.00,0.397517,0.000000,315.42,0.0,0.316798
2,0,55.00,0.359440,0.000000,354.55,0.0,0.339188
3,0,55.00,0.285707,0.000000,229.98,0.0,0.415428
4,0,55.00,0.101487,444.730391,307.12,0.0,0.569670
5,0,55.00,0.083655,92.284390,277.02,0.0,0.494765
6,0,55.00,0.107765,543.138844,690.38,0.0,0.522715
7,0,55.00,0.097857,578.379486,619.35,0.0,0.537815
8,0,55.75,0.179713,186.258945,539.15,0.0,0.547634
9,0,60.00,0.129596,1099.289203,608.28,0.0,0.438465


### Finding mean values of each period and building logistic regression model

In [13]:
tab_df_means = simp_tab_df.groupby([simp_tab_df['id']]).mean()
tab_df_means.reset_index(level=0, inplace=True)
# fill NA again to make the concept work
# (or not)
# tabular_df_means = tabular_df_means.fillna(axis='columns', method='bfill')
tab_df_means

,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,0,60.916667,0.238880,293.635812,430.134167,0.000000,0.445332
1,1,135.541667,0.060443,6940.579852,1119.352500,0.000000,0.350638
2,2,0.597500,0.574959,0.141369,4.168333,0.000000,0.875014
3,3,16.324112,357.412960,379.234397,19.188810,0.285477,21.883435
4,4,126.000000,0.133693,2568.283478,460.332500,0.000000,1.006855
...,...,...,...,...,...,...,...
5101,5101,94.166667,0.292280,180.142899,1218.388333,98.211165,0.465432
5102,5102,80.675000,0.755737,0.000000,215.499167,0.000000,0.772686
5103,5103,22.568333,0.801533,0.000000,145.231667,0.000000,0.851162
5104,5104,86.230833,0.068594,988.894377,72.641667,0.000000,0.548337


In [14]:
df_train = pd.merge(tab_df_means, train_df, on='id')
y = df_train['target']
X = df_train.drop(['id'], axis=1).fillna(method='bfill')

model = LogisticRegression(random_state=0, solver='newton-cg', multi_class='multinomial').fit(X, y)
model.score(X, y)

1.0

In [15]:
pred_df = pd.merge(tab_df_means, test_df, on='id')

X_pred = pred_df.drop(['id', 'score'], axis=1).fillna(method='bfill')

res = model.predict_proba(X_pred)[:,1]
res.round(6)
test_df.SCORE = res.round(6)

ValueError: X has 6 features per sample; expecting 7

In [ ]:
# export results to file
test_df.SCORE = res.round(6)
test_df.to_csv('HaidaTarasMonkaS_test.txt', sep=',', index=False)

In [61]:
simp_tab_df_groupped = simp_tab_df.groupby('id').agg(lambda x: list(x))
simp_tab_df_groupped.head(14)

,feature_1,feature_2,feature_3,feature_4
id,,,,
0,"[55.0, 110.0, 55.0, 55.0, 55.0, 55.0, 55.0, 55...","[0.43201678311238656, 0.3975167420697788, 0.35...","[0.0, 0.0, 0.0, 0.0, 444.730391, 92.28439, 543...","[176.78, 315.42, 354.55, 229.98, 307.12, 277.0..."
1,"[125.0, 125.0, 125.0, 125.0, 125.0, 126.5, 125...","[0.08317387509710256, 0.09683084061622006, 0.2...","[5893.939371, 5805.264756000001, 6696.44898400...","[812.2, 945.2, 837.6, 631.7, 1247.88, 982.82, ..."
2,"[1.17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0,...","[0.0, 1.696433, 0.0, 0.0, 0.0, 0.0, 5.20307313...","[0.0, 1.696433, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.05, 31.55, 0.0, 0.55, 0.0, 4.2, 0.18, 2.12,..."
3,"[33.5, 75.94, 78.06, 0.9793373684066452, 0.140...","[0.008239045952032066, 3747.802363, 532.755571...","[266.829103, 3747.802363, 532.755571, 0.979337...","[127.92, 92.25, 6.67, 0.9793373684066452, 0.14..."
4,"[103.0, 302.0, 110.0, 101.0, 126.0, 114.0, 115...","[0.16201509120499494, 0.1524421336062347, 0.17...","[4449.3348719999985, 5011.208142, 5491.642975,...","[535.37, 678.08, 824.85, 907.35, 359.7, 387.07..."
5,"[148.7, 145.0, 80.0, 195.0, 241.09, 115.0, 115...","[0.07256634027314493, 0.05486361204751952, 0.0...","[1082.113139, 3906.750156, 863.524972, 1743.89...","[490.33, 520.33, 395.7, 314.72, 479.42, 655.32..."
6,"[55.0, 115.0, 55.0, 57.0, 68.75, 55.0, 55.1, 5...","[0.29087475383551714, 0.2622024463773186, 0.29...","[0.0, 0.003, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[1742.93, 2118.63, 2751.43, 2429.13, 1933.43, ..."
7,"[95.0, 105.0, 95.0, 105.0, 98.0, 95.0, 95.0, 1...","[0.07887376832285656, 0.06567868184982993, 0.0...","[470.77933600000006, 491.352648, 378.25521, 39...","[1046.67, 1319.32, 1090.17, 1499.0, 1721.03, 9..."
8,"[0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0,...","[0.17218552094141568, 0.3146700443664416, 0.21...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.004695, 0.0, 0.0, ...","[0.67, 0.0, 0.12, 0.0, 0.55, 0.0, 0.0, 0.0, 0...."


### Create hash dictionary to encode them in the tabular data

In [6]:

# group/filter so unique hash names and create a 'dictionary' dataframe
unique_hashes_df = pd.DataFrame(hashed_df.feature_50.unique(), columns = ['feature_50'])

hashes_dict_df = unique_hashes_df
# create a new 'id' column, set 'feature_50' as a key and 'hash_id' as a value to for easier future replacement/matching
# drop index name and create a new 'id' column based on indexes

hashes_dict_df.reset_index(level=0, inplace=True)

hashes_dict_df = hashes_dict_df.set_index('feature_50')

hashes_dict_df.index.name = None
hashes_dict_df = hashes_dict_df.rename(columns = {'index': 'id'})


### Convert dictionary dataframe into actual python dictionary while also transposing it 
### It is needed for mapping them to replace hash codes with integer values - that can be used for model algorithm to evaluate them, it shouldn't be  possible with just hashes as strings (from what I understand)

In [7]:
hash_codes = hashes_dict_df.T.to_dict('list')

### Encode (map) main hash table with integer id's from above dictionary

In [8]:
hashed_df['feature_50'] = hashed_df['feature_50'].map(hash_codes)

# fucntion that is applied to convert 'feature_50' lists column into integers
def convert_to_int(x):
    return x[0]

hashed_df['feature_50'] = hashed_df['feature_50'].apply(convert_to_int)
print(hashed_df.head(3))

   id  feature_50
0   0           0
1   0           1
2   0           2


In [9]:
hashed_df_ = hashes_dict_df.T.to_dict('list')

## Restructure dataframe to put 12 periods in 1 data cell

In [13]:
tabular_df.head()

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,1,110,55,0.432017,0,176.78,0,0.323712,147,...,0,0,0,0,55,2,0.526552,145,133.28,0
1,0,2,110,110,0.397517,0,315.42,0,0.316798,135,...,0,0,0,0,110,1,0.481063,130,229.97,0
2,0,3,110,55,0.35944,0,354.55,0,0.339188,181,...,0.07,0,0,0,55,1,0.509598,180,231.78,0
3,0,4,110,55,0.285707,0,229.98,0,0.415428,142,...,0,0,0,0,55,0,0.680089,142,183.83,0
4,0,5,110,55,0.101487,444.73,307.12,0,0.56967,92,...,0.95,0,20.0145,0,55,0,0.776175,85,155.83,0


In [20]:
new_tab_df = tabular_df.groupby('id')['feature_0'].apply(list)


## Find mean values of 12 periods

In [11]:
tabular_df_means = tabular_df.groupby([tabular_df['id']]).mean()
tabular_df_means.reset_index(level=0, inplace=True)
# fill NA again to make the concept work
tabular_df_means = tabular_df_means.fillna(axis='columns', method='bfill')
tabular_df_means

DataError: No numeric types to aggregate

## Buildign logistic regression model without hashes

In [ ]:
df_train = pd.merge(tabular_df_means, train_df, on='id')
y = df_train.target
X = df_train.drop(['period', 'id', 'target'], axis=1).fillna(method='bfill')

model = LogisticRegression(random_state=0, solver='newton-cg', multi_class='multinomial').fit(X, y)
model.score(X, y)

## Predicting/testing

In [ ]:
test_df_n = pd.merge(tabular_df_means, test_df, on='id')

X_pred = test_df_n.drop(['period', 'id', 'score'], axis=1).fillna(method='bfill')

res = model.predict_proba(X_pred)[:,1]
res.round(6)
test_df.SCORE = res.round(6)

## Export final result to file

In [ ]:
test_df.SCORE = res.round(6)
test_df.to_csv('HaidaTarasMonkaS_test.txt', sep=',', index=False)